In [1]:
%matplotlib inline
import tensorflow as tf
from tfutils import base, data, model, optimizer, utils
import numpy as np
import matplotlib.pyplot as plt
import sys
from dldata.metrics.utils import compute_metric_base

Can't import separate mcc package
Can't import asgd.
Can't import scikits stuff


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
"""
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
images_array = inputs['images'].eval(sess)
sess.close()
"""

In [ ]:
# plt.imshow(images_array[0,::]) # example image plot

## Loading Database

In [ ]:
import pymongo as pm
port = 24444
dbname = 'assignment2'
collection = 'alexnet.files'
exp_id = '1st_experiment'

connection = pm.MongoClient(port=port)
connection.database_names()
db = connection[dbname]
col1 = db[collection]
exp_out = list(col1.find({'exp_id': exp_id}))
validation_results = list(col1.find({'validation_results' : {'$exists' : True}}))

In [ ]:
training_events = list(col1.find({'exp_id': exp_id, 'validation_results' : {'$exists' : False}}))
train_results = [i for sublist in training_events for i in sublist['train_results']]

In [ ]:
# plotting loss
plt.plot([i['loss'] for i in train_results])

## Testing VAE

In [5]:
sys.path.append('assignment2')

from VAE_models import VAE, get_conv

# test images
n_images = 100
resolution = 24
channel = 3
images = tf.Variable(tf.truncated_normal([n_images,resolution,resolution,channel]))
labels = np.random.permutation(range(n_images))
inputs = {'images': images,
         'labels': labels}

#model_layers, _ = VAE(inputs)
filters = [[4,4,3,64], [4,4,64,128]]
with tf.variable_scope('conv11'):
        conv1, conv1_weights = get_conv(inputs['images'], filters[0], 2)
with tf.variable_scope('conv21'):
    conv2, conv2_weights = get_conv(conv1, filters[1], 2)

ValueError: Dimensions must be equal, but are 128 and 64 for 'conv21/BiasAdd' (op: 'BiasAdd') with input shapes: [100,6,6,128], [64].

In [ ]:
def get_FC(inputs, shape, activation=tf.nn.tanh):
    """
    Args:
        shape: [input, output]
    """
    weights = tf.get_variable('weights', shape, tf.float32, tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('biases', [shape[1]], tf.float32, tf.zeros_initializer())
    FC = tf.nn.xw_plus_b(inputs, weights, biases, name='FC')
    if activation is not None:
        out = activation(FC)
    else:
        out = FC
    return out

n_hidden = 10; n_latent = 20
outputs = {}
# flatten the input images
inp = tf.reshape(inputs['images'], [inputs['images'].get_shape().as_list()[0], -1])
flat_len = inp.shape.as_list()[1]
# encoder
with tf.variable_scope('latent'):
    latent = get_FC(inp, [flat_len, n_hidden])
with tf.variable_scope('mu'):
    mu = get_FC(latent, [n_hidden, n_latent], activation=None)
with tf.variable_scope('logstd'):
    logstd = get_FC(latent, [n_hidden, n_latent], activation=None)
outputs['latent'] = latent
outputs['mu'] = mu
outputs['logstd'] = logstd

# magic reparameterization trick to the rescue!
noise = tf.random_normal([1, n_latent])
z = tf.add(mu, tf.multiply(noise, tf.exp(.5*logstd)), name='latent_encoding') # where the magic happens

# decoder
with tf.variable_scope('hidden_out'):
    hidden_out = get_FC(z, [n_latent, n_hidden])
with tf.variable_scope('reconstruction'):
    reconstruction = get_FC(hidden_out, [n_hidden, flat_len], tf.sigmoid)
outputs['hidden_out'] = hidden_out
outputs['reconstruction'] = reconstruction


In [ ]:
model_layers

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
predictions = sess.run(out)
sess.close()
predictions.shape

In [ ]:
# in_top_k
pred = np.array([[.2,.5,.3],[.7,.1,.1]])
targets = np.array([1,1])
func = tf.nn.in_top_k(pred,targets,1)

sess = tf.Session()
sess.run(func)
sess.close()

## Testing gabors

In [ ]:
%matplotlib inline
from skimage.filters import gabor_kernel
from skimage import io
from matplotlib import pyplot as plt  
import math

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

size = np.linspace(-np.pi,np.pi,200)
xgrid, ygrid = np.meshgrid(size, size)

gaussian = np.exp(-(xgrid/2)**2-(ygrid/2)**2) 
sine = np.sin(xgrid*10)
gabor = sine * gaussian
plt.imshow(gabor,cm.gray) 
plt.show()

In [ ]:
np.arange(0,2*math.pi,0.1)

In [ ]:
import pymongo as pm
connection = pm.MongoClient(port=24444)
connection.database_names()
db = connection['imagenet']
col1 = db['alexnet']
exp_out = list(col1.find({'exp_id': 'experiment_1'}))
exp_out

In [ ]:
exp_out